# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
type(stocks)

pandas.core.frame.DataFrame

In [3]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [6]:
adjective = 'awesome'
string = f'freecodeCamp is {adjective}'
print(string)

freecodeCamp is awesome


In [14]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 92741786, 'calculationPrice': 'tops', 'change': 0.48, 'changePercent': 0.00347, 'close': 0, 'closeSource': 'ifofilca', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': 'eeErc alptim X Iire', 'highTime': 1733129964511, 'iexAskPrice': 144.86, 'iexAskSize': 103, 'iexBidPrice': 139.78, 'iexBidSize': 510, 'iexClose': 143.98, 'iexCloseTime': 1693569055870, 'iexLastUpdated': 1743566012725, 'iexMarketPercent': 0.025939930522567933, 'iexOpen': 137.34, 'iexOpenTime': 1735515963332, 'iexRealtimePrice': 143.66, 'iexRealtimeSize': 104, 'iexVolume': 1727719, 'lastTradeTime': 1730119079431, 'latestPrice': 139.85, 'latestSource': 'IEX real time price', 'latestTime': '3:24:57 PM', 'latestUpdate': 1744803603380, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2

In [15]:
data['symbol']

'AAPL'

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [17]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [18]:
column = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = column)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [22]:
final_dataframe.append(pd.Series([symbol, price, market_cap,'N/A'],index = column),ignore_index=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_15116\310326520.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(pd.Series([symbol, price, market_cap,'N/A'],index = column),ignore_index=True)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,139.85,2261675342301,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [51]:
final_dataframe = pd.DataFrame(columns = column)

In [70]:
#135, 219, 467, 484
for stock in stocks['Ticker'][485:]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(pd.Series([stock, data['latestPrice'], data['marketCap'],'N/A'],index = column),ignore_index=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_15116\2163209206.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(pd.Series([stock, data['latestPrice'], data['marketCap'],'N/A'],index = column),ignore_index=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_15116\2163209206.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(pd.Series([stock, data['latestPrice'], data['marketCap'],'N/A'],index = column),ignore_index=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_15116\2163209206.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(pd.Series([stock, data['latestPrice'], data['marketCap'],'N/A'],index = colu

In [96]:
# DISCA has been Ticker delisted
#api_url = 'https://sandbox.iexapis.com/stable/stock/'+stocks['Ticker'][135]+'/quote/?token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(api_url).json()
#final_dataframe = final_dataframe.append(pd.Series([stocks['Ticker'][135], data['latestPrice'], data['marketCap'],'N/A'],index = column),ignore_index=True)

In [97]:
# HFC 
#api_url = 'https://sandbox.iexapis.com/stable/stock/'+stocks['Ticker'][219]+'/quote/?token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(api_url).json()
#final_dataframe = final_dataframe.append(pd.Series([stocks['Ticker'][219], data['latestPrice'], data['marketCap'],'N/A'],index = column),ignore_index=True)

In [98]:
# VIAC:AV has changed to a new ticker symbol: 2341170Z:US
#api_url = 'https://sandbox.iexapis.com/stable/stock/'+stocks['Ticker'][467]+'/quote/?token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(api_url).json()
#final_dataframe = final_dataframe.append(pd.Series([stocks['Ticker'][467], data['latestPrice'], data['marketCap'],'N/A'],index = column),ignore_index=True)

In [99]:
# WLTW is changed WTW:US
#api_url = 'https://sandbox.iexapis.com/stable/stock/'+'WTW:US'+'/quote/?token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(api_url).json()
#final_dataframe = final_dataframe.append(pd.Series(['WTW:US', data['latestPrice'], data['marketCap'],'N/A'],index = column),ignore_index=True)

In [95]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,142.48,42279335397,N/A
1,AAL,14.36,9668407495,N/A
2,AAP,184.19,11313748455,N/A
3,AAPL,139.37,2230425072464,N/A
4,ABBV,148.5,265935466814,N/A
...,...,...,...,...
496,YUM,127.14,36023667026,N/A
497,ZBH,108.3,23422337266,N/A
498,ZBRA,241.7,12612685380,N/A
499,ZION,51.984,7479573949,N/A


In [90]:
# Saving model to a csv file named 'stocks.csv'
final_dataframe.to_csv('stocks.csv')

In [86]:
stocks['Ticker'][484]

'WLTW'

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: